# Extract data from output files
### Analyze the output from a single LBANN run
March 9, 2020 \
April 6, 2020 : to store files in order of epochs \
April 21, 2020: added jupyter widgets to compare pixel intensity plots \
July 30, 2020: Perform analysis with stored histograms.

In [26]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import subprocess as sp
import os
import glob
import sys

import itertools
import time

from ipywidgets import *

In [27]:
%matplotlib widget

In [28]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis')
from modules_image_analysis import *

In [29]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4. + 1e-8) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s + 1e-8)

## Extract image data 

## Read dataframe

In [30]:
dict1={'scratch':'/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/512square/',
    'proj':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/'}

u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
display(u)


interactive(children=(Select(description='x', options=('scratch', 'proj'), value='scratch'), Output()), _dom_c…

In [54]:
parent_dir=u.result
dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'20*')]
w=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(w)

interactive(children=(Dropdown(description='x', options=('20200901_200005_bsize50_30eph', '20200905_080143_bsi…

In [143]:
result=w.result
main_dir=parent_dir+result
print(main_dir)

/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/512square/20200905_080143_bsize25_test


In [144]:
# main_dir=''

In [145]:
### Load data
df=pd.read_pickle(main_dir+'/df_processed.pkle')
df[['epoch','step']]=df[['epoch','step']].astype(int)
df['label']=df.epoch.astype(str)+'-'+df.step.astype(str) # Add label column for plotting

In [146]:
print(df.shape)
df.head()


(36000, 20)


,epoch,step,img_type,fname,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,chi_spec1,chi_spec2,num_large,num_vlarge,hist_val,hist_err,hist_bin_centers,spec_val,spec_err,label
0,0,0,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,156.365736,0.571041,0.013389,156.950166,11.801882,94.639299,130752.434132,1.573505e+08,0,0,"[4.325866710376738e-05, 0.04932174690214731, 0...","[1.0038103618348732e-05, 0.002557320321533822,...","[-1.031746031584782, -0.6161616169043975, -0.3...","[15816745510.91095, 58745.21193428039, 29045.9...","[23733321.563611772, 2795.798160994336, 987.21...",0-0
1,0,1,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,153.266864,0.809436,0.013389,154.089688,11.572906,120.313011,142924.512667,1.140362e+08,0,0,"[0.00020067215017580982, 0.0650798750926199, 0...","[3.803333405741826e-05, 0.0034644261587476652,...","[-1.031746031584782, -0.6161616169043975, -0.3...","[15896345649.638384, 55475.957067871095, 27286...","[30540379.2306359, 2749.5342099141385, 1360.61...",0-1
2,0,2,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,152.217309,0.878861,0.013389,153.109558,11.397798,133.295394,161520.861279,1.260810e+08,0,0,"[0.0007657384892187711, 0.08708252920213547, 0...","[0.00012294591578446444, 0.004126784730731057,...","[-1.031746031584782, -0.6161616169043975, -0.3...","[15996659935.473005, 47554.68747327805, 24018....","[31103667.63689838, 2101.7693819087235, 1126.4...",0-2
3,0,3,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,143.842021,0.638344,0.013389,144.493753,11.148249,144.079024,262057.872702,1.573687e+08,0,0,"[0.0007420063037937878, 0.0913547517276373, 0....","[0.0001507374819746063, 0.005366789951150719, ...","[-1.031746031584782, -0.6161616169043975, -0.3...","[16339795966.294155, 21653.76312286377, 14125....","[19674398.376583222, 1278.9793959454978, 572.4...",0-3
4,0,4,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,136.889605,0.546673,0.013389,137.449667,10.762711,98.156824,436560.938669,3.476488e+08,0,0,"[0.0006918382662498354, 0.10640052811800137, 0...","[0.00011421355422153463, 0.004650850716565966,...","[-1.031746031584782, -0.6161616169043975, -0.3...","[16675184403.206768, 11503.126218414307, 9353....","[3525154.8964999802, 970.9560531187026, 446.94...",0-4


In [107]:
df.tail(5)

,epoch,step,img_type,fname,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,chi_spec1,chi_spec2,num_large,num_vlarge,hist_val,hist_err,hist_bin_centers,spec_val,spec_err,label
8995,39,8995,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.124948,0.060452,0.010527,0.195928,0.122717,12.513856,1.988131,13610.875362,3,0,"[1.317291000902962, 0.6720909415107557, 0.2353...","[0.004100088342618162, 0.003855656836683521, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[53620452355.92751, 1604316.7863863374, 136847...","[74686953.8757033, 79527.45658135906, 45708.78...",39-8995
8996,39,8996,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.130391,0.065918,0.008353,0.204662,0.126812,12.619687,2.900195,15776.287550,0,0,"[1.3224076901213295, 0.6689307950310496, 0.233...","[0.004119565642596587, 0.003756035168701094, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[53732171067.574646, 1521814.4944151687, 13221...","[76864135.24080047, 69545.58462315932, 53152.0...",39-8996
8997,39,8997,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.126041,0.063967,0.009030,0.199038,0.123386,11.422446,3.056611,20510.425989,2,0,"[1.318628564897959, 0.6720147024015991, 0.2343...","[0.0036318366748270157, 0.0034686095861014947,...","[-1.031746031584782, -0.6161616169043975, -0.3...","[53652529861.06401, 1573983.4778804015, 131677...","[64941241.8083904, 78379.40465781574, 52823.01...",39-8997
8998,39,8998,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.110288,0.051755,0.013519,0.175561,0.109104,14.159677,2.747784,15404.878199,3,0,"[1.3019644401277264, 0.6852980099743982, 0.241...","[0.0044430593327513795, 0.003803412175981078, ...","[-1.031746031584782, -0.6161616169043975, -0.3...","[53318921316.65061, 1562581.3738323974, 132731...","[85307834.84354825, 76232.77376926558, 52915.7...",39-8998
8999,39,8999,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.110634,0.051448,0.009478,0.171559,0.109596,11.937854,2.789505,17865.138178,3,0,"[1.3040491924543474, 0.6858910463885042, 0.239...","[0.0035824893437912545, 0.003204497389979377, ...","[-1.031746031584782, -0.6161616169043975, -0.3...","[53358365589.39348, 1460182.2749747848, 131527...","[67702225.31073514, 70169.35138223138, 57084.4...",39-8999


In [108]:
df.step.values

array([   0,    1,    2, ..., 8997, 8998, 8999])

### View best epochs

#### Locations with best chi_sqr

In [109]:
chi_sqr_keys=['epoch','step','chi_1a','chi_1b','chi_1c','chi_1','chi_2','chi_imgvar','chi_spec1','chi_spec2']
# index location of min/max values of chi squares
inds=[]
for key in ['chi_1a','chi_1b','chi_1c','chi_1','chi_2','chi_imgvar','chi_spec1','chi_spec2']:
    inds.append(df[key].idxmin(axis=1))
inds=np.array(inds)
df.loc[inds][chi_sqr_keys]

,epoch,step,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,chi_spec1,chi_spec2
5515,24,5515,0.004034,0.022230,0.853907,0.880171,0.000944,19.813158,6.519444,6.144320e+03
2221,9,2221,0.228698,0.000159,5.418209,5.647067,0.215282,40.374502,1.316795,1.319976e+03
4119,18,4119,0.512681,0.001410,0.001059,0.515149,0.466163,40.070547,1844.483660,6.065205e+06
2457,10,2457,0.019299,0.055117,0.023113,0.097529,0.005946,30.638395,7.267183,9.503446e+03
5515,24,5515,0.004034,0.022230,0.853907,0.880171,0.000944,19.813158,6.519444,6.144320e+03
4127,18,4127,0.460782,0.187034,0.013389,0.661205,0.428990,3.988882,7847.002214,5.725329e+07
8174,36,8174,0.055167,0.059600,0.023367,0.138133,0.049574,11.980135,0.243944,1.631562e+03
2768,12,2768,0.044075,0.003920,24.010897,24.058892,0.041496,47.098375,0.527854,5.114069e+02


In [110]:
def f_get_best_chisqr_models(df,cutoff=0.2):
    '''
    Pick models with lowest 20% chi-square for multiple categories
    '''
    chi_sqr_keys=[ 'chi_1', 'chi_spec1']
    q_dict=dict(df.quantile(q=cutoff,axis=0)[chi_sqr_keys])
#     print(q_dict)
    
    df_sliced=df.query('chi_1 < {0} & chi_spec1 < {1}'.format(q_dict['chi_1'],q_dict['chi_spec1']))
    
    return df_sliced

In [111]:
df_sliced=f_get_best_chisqr_models(df,cutoff=0.3)
print(df_sliced.shape)

(895, 20)


In [112]:
df_sliced.sort_values(by=['chi_1','chi_spec1'])[['epoch','step','chi_1','chi_spec1']].head(5)

,epoch,step,chi_1,chi_spec1
2280,10,2280,0.109835,1.462857
8175,36,8175,0.111891,0.419815
8176,36,8176,0.122998,0.523242
2278,10,2278,0.123678,2.182172
8380,37,8380,0.123990,1.422348


In [113]:
### Plot chi-sqr values
df_sliced.plot(x="step", y=["chi_1", "chi_imgvar", "chi_spec1"],style='.',marker='*')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### High Pixel images

In [114]:
### Plot number of very high pixel images
plt.figure()
plt.plot(df[df.img_type=='train_gen'].step,df[df.img_type=='train_gen'].num_vlarge,linestyle='',marker='*')
plt.xlabel('Steps in Epochs')
plt.ylabel('Number of very large pixel images from a batch of images')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Number of very large pixel images from a batch of images')

In [115]:
df[df.num_vlarge>10].tail()

,epoch,step,img_type,fname,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,chi_spec1,chi_spec2,num_large,num_vlarge,hist_val,hist_err,hist_bin_centers,spec_val,spec_err,label
8788,39,8788,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.322140,0.254744,44.283180,44.860064,0.161013,29.303902,6.497325,16937.626534,80,27,"[1.292059018148184, 0.6129472257066492, 0.2517...","[0.004246021305834264, 0.002150469242716277, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[52031066396.67862, 4658871.906463165, 3681401...","[115702605.90797107, 225309.11292546676, 13530...",39-8788
8789,39,8789,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.327436,0.263085,58.894415,59.484936,0.154746,29.377669,6.828549,19762.143868,85,27,"[1.2841001566569898, 0.6192791559177647, 0.255...","[0.004212209451800178, 0.002138521985090173, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[51868808009.040504, 4488173.086867218, 370565...","[114347318.4342004, 223334.42308592927, 125828...",39-8789
8790,39,8790,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.323601,0.254464,62.718238,63.296303,0.147551,34.420501,6.742242,15659.298041,81,29,"[1.2768133797654226, 0.6268300636563238, 0.258...","[0.005207553980196202, 0.002642385112993198, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[51753894077.57866, 4140167.608957634, 3496907...","[139269316.95312038, 199102.25111686118, 13018...",39-8790
8791,39,8791,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.272585,0.155086,34.001333,34.429003,0.143544,30.513890,5.808251,14352.839633,82,20,"[1.2845400794225108, 0.630785312788313, 0.2551...","[0.00522811665072242, 0.0030339453416371115, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[52066399963.96625, 4605015.042946167, 3430008...","[133055120.46672511, 235988.91195383467, 11626...",39-8791
8792,39,8792,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.237371,0.101234,22.866700,23.205304,0.139502,32.731646,4.845675,9883.774207,72,22,"[1.2885297618925307, 0.6356359154797065, 0.252...","[0.005964142483214694, 0.0035760898146968076, ...","[-1.031746031584782, -0.6161616169043975, -0.3...","[52259726368.40461, 3989609.4505187226, 331200...","[148853409.7927219, 208830.15077236434, 140209...",39-8792


## Compare samples

In [116]:
# df_sliced.sort_values(by=['chi_spec1','chi_1']).head(5)
# df_sliced.sort_values(by=['chi_1','chi_spec1']).head(5)

In [147]:
df_best=df[df.step%900==0].sort_values(by=['chi_spec1','chi_1']).head(5)
# df_best=df[df.step%225==0].sort_values(by=['chi_1','chi_spec1']).head(5)

df_best

,epoch,step,img_type,fname,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,chi_spec1,chi_spec2,num_large,num_vlarge,hist_val,hist_err,hist_bin_centers,spec_val,spec_err,label
20700,23,20700,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.002152,0.006222,0.133078,0.141451,0.001909,30.291478,0.122206,115.979712,6,0,"[1.1394468098467028, 0.9215429463276906, 0.264...","[0.006976134583175107, 0.004439174845834915, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[50946331550.587234, 2348159.966225586, 260083...","[147808864.16813678, 169497.98261407675, 19200...",23-20700
17100,19,17100,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.025808,0.035041,0.019616,0.080464,0.023525,15.638320,0.143824,519.411617,2,0,"[1.2062496073365163, 0.8345118726233761, 0.254...","[0.003984121423641599, 0.003596366292204255, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[52014202541.318085, 2473222.548847656, 199668...","[68894637.70126398, 197864.35593253368, 122352...",19-17100
18000,20,18000,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.021419,0.017284,0.015893,0.054596,0.021137,21.189591,0.166350,472.503121,1,0,"[1.1960261423443261, 0.8362353957659847, 0.258...","[0.005326972816520755, 0.004567919654161823, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[51709953217.43758, 2694231.7470703125, 252396...","[101331733.16392693, 305654.01795225154, 19677...",20-18000
19800,22,19800,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.029435,0.006725,0.182625,0.218785,0.028557,21.214182,0.171117,369.663988,6,0,"[1.2126502714286087, 0.8187595336598573, 0.247...","[0.004522757762263888, 0.006307993155964922, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[51841780107.883125, 2198229.7688671877, 20485...","[56359441.76584883, 209921.11842454752, 143795...",22-19800
30600,34,30600,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.001382,0.000612,0.081859,0.083853,0.001139,30.949352,0.181713,126.377551,1,0,"[1.1388321762849367, 0.9319709458402139, 0.255...","[0.007316995192709413, 0.004632295332949317, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[50872762301.91508, 2065442.2685351563, 188049...","[148602106.29350153, 179117.06002432897, 10495...",34-30600


In [119]:
# best_idx=[]
# best_idx.append(df_sliced.sort_values(by=['chi_1','chi_spec1']).head(2).index)
# best_idx.append(df_sliced.sort_values(by=['chi_spec1','chi_1']).head(2).index)

# best_idx=[i for j in best_idx for i in j]
# print(best_idx)

# df_best=df_sliced.loc[best_idx]
# print([(df_best.loc[idx].epoch,df_best.loc[idx].step) for idx in best_idx])
# df_best

[2280, 8175, 8174, 8173]
[(10, 2280), (36, 8175), (36, 8174), (36, 8173)]


,epoch,step,img_type,fname,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,chi_spec1,chi_spec2,num_large,num_vlarge,hist_val,hist_err,hist_bin_centers,spec_val,spec_err,label
2280,10,2280,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.024631,0.052044,0.033160,0.109835,0.010957,19.427662,1.462857,6537.345726,10,1,"[1.1451939143573837, 0.8738484879887877, 0.295...","[0.005272387236015928, 0.0031869939790779727, ...","[-1.031746031584782, -0.6161616169043975, -0.3...","[50979378477.40453, 2188314.5014541624, 185563...","[109837225.11543675, 117196.58049777108, 94842...",10-2280
8175,36,8175,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.059576,0.039340,0.012975,0.111891,0.041533,12.564539,0.419815,2131.218816,6,1,"[1.2038490698362712, 0.780621856308551, 0.2844...","[0.003719119974952704, 0.0026574458016790188, ...","[-1.031746031584782, -0.6161616169043975, -0.3...","[51683678432.22983, 3039934.1436807252, 215903...","[73658154.43771079, 99173.3975615759, 57394.56...",36-8175
8174,36,8174,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.055167,0.059600,0.023367,0.138133,0.049574,11.980135,0.243944,1631.562085,5,1,"[1.230118801340549, 0.7691016049646534, 0.2696...","[0.003574968088975311, 0.002616131871867256, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[52249120171.61925, 3063728.3414559937, 205122...","[70529582.6240571, 103985.02889599626, 60611.5...",36-8174
8173,36,8173,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.055482,0.066679,0.009286,0.131448,0.051135,12.635242,0.268307,1515.055722,2,0,"[1.2370930369815027, 0.7687575292508323, 0.264...","[0.0039740893388466765, 0.002858510028283685, ...","[-1.031746031584782, -0.6161616169043975, -0.3...","[52414246543.75439, 3049092.24581604, 2008731....","[78481610.97135213, 91095.73545576222, 50876.1...",36-8173


## Plot pixel intensity and spectrum

In [82]:
def f_plot_hist_spec_combined(df,dict_bkg):

    fig=plt.figure(figsize=(6,6))
    ax1=fig.add_subplot(121)
    ax2=fig.add_subplot(122)
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):

        x1=row.hist_bin_centers
        y1=row.hist_val
        yerr1=row.hist_err
        x1=f_invtransform(x1)

        y2=row.spec_val
        yerr2=row.spec_err
        x2=np.arange(len(y2))

        label='{0}_{1}_{2}'.format(i,row.epoch,row.step)
        ax1.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
    #     ax2.errorbar(x2,y2,yerr2,marker=marker,markersize=5,linestyle='',label='{0}-{1}'.format(epoch,step))

        ax2.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
        ax2.plot(x2, y2, marker=marker, linestyle=':',label=label)

    ### Plot input data
    x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
    x=f_invtransform(x)
    ax1.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   

    y,yerr=dict_bkg['spec_val'],dict_bkg['spec_err']
    x=np.arange(len(y))
    ax2.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)

    plt.legend()
    # plt.yscale('log')
    ax1.set_xscale('symlog',linthreshx=50)
    ax1.set_yscale('log')
    ax2.set_yscale('log')
    
    

def f_plot_hist_spec_best(df,dict_bkg,plot_type):

    fig=plt.figure(figsize=(6,6))
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y2=row.spec_val
            yerr2=row.spec_err
            x2=np.arange(len(y2))
            
            plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
            plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

    ### Plot input data
    if plot_type=='hist':
        x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
        x=f_invtransform(x)
        plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
        plt.title('Pixel Intensity Histogram')
        plt.xscale('symlog',linthreshx=50)
        
    if plot_type=='spec':
        y,yerr=dict_bkg['spec_val'],dict_bkg['spec_err']
        x=np.arange(len(y))
        plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
        plt.title('Spectrum')
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
    plt.yscale('log')
    

In [83]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_err=f_compute_spectrum(sample,plot=False)
    
    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_err':spec_err }
    return dict1


### Extract validation data
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/512_square/dataset1_smoothing_single_universe/norm_1_train_val.npy'
s_val=np.load(fname,mmap_mode='r')[:1000][:,0,:,:]
print(s_val.shape)

bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1) 
# Compute histogram and spectrum of raw data 
dict_val=f_compute_hist_spect(s_val,bins)


(1000, 512, 512)


In [148]:
f_plot_hist_spec_combined(df_best,dict_val)
# f_plot_hist_spec_combined(df[df.epoch==10].head(10),dict_val)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [149]:
f_plot_hist_spec_best(df_best,dict_val,'spec')
f_plot_hist_spec_best(df_best,dict_val,'hist')

/global/homes/v/vpa/.conda/envs/v_py3/lib/python3.6/site-packages/ipykernel_launcher.py:43: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/global/homes/v/vpa/.conda/envs/v_py3/lib/python3.6/site-packages/ipykernel_launcher.py:43: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## View image block

In [27]:
def f_get_img(df,epoch,step):
    df_temp=df[(df.epoch==epoch)&(df.step==step)]
    images=np.load(df_temp.fname.values[0])[:,0,:,:]
    return images

img_arr=f_get_img(df,24,10836)

In [28]:
f_plot_grid(img_arr[:18],cols=6,fig_size=(10,6))

3 6


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [29]:
f_plot_grid(s_val[100:118],cols=6,fig_size=(10,6))

3 6


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:
# df_sliced[df_sliced.epoch==4]['step'].values

In [33]:
# df[(df.epoch==18)&(df.step>=15975)&(df.step<=15983)][['epoch','step','chi_1','chi_spec1','num_vlarge']]